In [1]:
import pyomo.environ as pyo
from operator import mul
from pyomo.environ import *
m= pyo.ConcreteModel()

In [2]:
from Parameters import * 
db_connection = mysql.connector.connect(
host="localhost",
user="root",
passwd="root", # change password
auth_plugin='mysql_native_password'
)
db_cursor = db_connection.cursor(buffered=True)
db_cursor.execute("USE earthquake")

In [3]:
K = Parameters.num_of_immediate(db_cursor,0,200) #number of victims with condition 3 (immediate)
L = Parameters.num_of_delayed(db_cursor,0,200) #number of victims condition 2 (delay)
H = Parameters.num_of_hospitals(db_cursor) #operational hospitals
victim_dict = Parameters.victim_dict(db_cursor,0,200) #all victims
immediate_v_id_list, delayed_v_id_list = Parameters.victim_id_lists(victim_dict) #immediate and delayed victim id's
hospital_dict = Parameters.hospital_dict(db_cursor) 
hospital_id_list = Parameters.hospital_id_lists(hospital_dict) 
neighbourhood_list, hospital_list, distances = Parameters.distances(victim_dict, hospital_dict)
bed_capacity_list = Parameters.hospital_bed_capacity_lists(hospital_dict)
y_immediate_list, y_delayed_list = Parameters.y_lists(victim_dict) #age
t_immediate_list, t_delayed_list = Parameters.t_lists(victim_dict) #time elapsed
neighbourhood_immediate_list, neighbourhood_delayed_list = Parameters.neighbourhood_lists(victim_dict) 

##### Variables

In [4]:
#x: Binary variable. victim i to hospital j
m.I = pyo.RangeSet(0, len(immediate_v_id_list)-1)
m.D = pyo.RangeSet(0, len(delayed_v_id_list)-1)
m.J = pyo.RangeSet(0, len(hospital_id_list)-1)

m.K = np.ndarray(shape=(len(immediate_v_id_list),len(hospital_id_list)))
m.L = np.ndarray(shape=(len(delayed_v_id_list),len(hospital_id_list)))

def IJ_rule(m):
    return [(i,j) for i in range(len(m.I)) for j in range(len(m.K[i]))]

def DJ_rule(m):
    return [(d,j) for d in range(len(m.D)) for j in range(len(m.L[d]))]

m.immediate = Set(within=m.I*m.J, initialize=IJ_rule)
m.immediate_v = Var(m.immediate, within = Binary) #for immediate
m.delayed = Set(within=m.D*m.J, initialize=DJ_rule)
m.delayed_v = Var(m.delayed, within = Binary) #for delayed

##### Parameters

In [5]:
c_1 = 1.001 #condition: immediate
c_2 = 1 #condition: delayed
#epsilon value is calculated as the average of the coefficients of the x_ij's in order to make the penalty meaningful
E = (c_2*sum(map(mul, y_immediate_list, t_immediate_list)) + c_1*sum(map(mul, y_delayed_list, t_delayed_list)))/(c_1+c_2)

##### Objective Function 


In [6]:
def obj_rule(m):
    immediate = 0
    for k in range(K):
        for j in range(H):
            neighbourhood_index = neighbourhood_list.index(neighbourhood_immediate_list[k])
            immediate += c_1*t_immediate_list[k]*y_immediate_list[k]*distances[neighbourhood_index][j]*m.immediate_v[k,j]
            
    delayed = 0
    for l in range(L):
        for j in range(H):
            neighbourhood_index = neighbourhood_list.index(neighbourhood_delayed_list[l])
            delayed += c_2*t_delayed_list[l]*y_delayed_list[l]*distances[neighbourhood_index][j]*m.delayed_v[l,j]
            
    penalty_immediate = K - sum(m.immediate_v[k,j] for k in range(K) for j in range(H))
    
    penalty_delayed = L - sum(m.delayed_v[l,j] for l in range(L) for j in range(H))
    
    return immediate + delayed + E*(c_1*penalty_immediate + c_2*penalty_delayed)

m.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

##### Constraints

In [7]:
def bed_capacity_constraint(m, j): 
    return sum(m.immediate_v[k, j] for k in range(K)) + sum(m.delayed_v[l, j] for l in range(L)) <= bed_capacity_list[j]

m.bed_capacity_constraint = pyo.Constraint(range(H), rule = bed_capacity_constraint)

In [8]:
def allocation_constraint_immediate(m, k): 
    return sum(m.immediate_v[k, j] for j in range(H) ) <= 1 
m.allocation_constraint_immediate = pyo.Constraint(range(K), rule = allocation_constraint_immediate)

def allocation_constraint_delayed(m, l): 
    return sum(m.delayed_v[l, j] for j in range(H) ) <= 1 
m.allocation_constraint_delayed = pyo.Constraint(range(L), rule = allocation_constraint_delayed)

In [9]:
solver = SolverFactory('glpk')
solution = solver.solve(m)
m.display()

Model unknown

  Variables:
    immediate_v : Size=2744, Index=immediate
        Key      : Lower : Value : Upper : Fixed : Stale : Domain
          (0, 0) :     0 :   0.0 :     1 : False : False : Binary
          (0, 1) :     0 :   0.0 :     1 : False : False : Binary
          (0, 2) :     0 :   1.0 :     1 : False : False : Binary
          (0, 3) :     0 :   0.0 :     1 : False : False : Binary
          (0, 4) :     0 :   0.0 :     1 : False : False : Binary
          (0, 5) :     0 :   0.0 :     1 : False : False : Binary
          (0, 6) :     0 :   0.0 :     1 : False : False : Binary
          (0, 7) :     0 :   0.0 :     1 : False : False : Binary
          (1, 0) :     0 :   0.0 :     1 : False : False : Binary
          (1, 1) :     0 :   0.0 :     1 : False : False : Binary
          (1, 2) :     0 :   0.0 :     1 : False : False : Binary
          (1, 3) :     0 :   0.0 :     1 : False : False : Binary
          (1, 4) :     0 :   0.0 :     1 : False : False : Binary
   

        Key       : Lower : Value : Upper : Fixed : Stale : Domain
           (0, 0) :     0 :   0.0 :     1 : False : False : Binary
           (0, 1) :     0 :   0.0 :     1 : False : False : Binary
           (0, 2) :     0 :   0.0 :     1 : False : False : Binary
           (0, 3) :     0 :   0.0 :     1 : False : False : Binary
           (0, 4) :     0 :   0.0 :     1 : False : False : Binary
           (0, 5) :     0 :   0.0 :     1 : False : False : Binary
           (0, 6) :     0 :   0.0 :     1 : False : False : Binary
           (0, 7) :     0 :   0.0 :     1 : False : False : Binary
           (1, 0) :     0 :   0.0 :     1 : False : False : Binary
           (1, 1) :     0 :   0.0 :     1 : False : False : Binary
           (1, 2) :     0 :   0.0 :     1 : False : False : Binary
           (1, 3) :     0 :   0.0 :     1 : False : False : Binary
           (1, 4) :     0 :   0.0 :     1 : False : False : Binary
           (1, 5) :     0 :   0.0 :     1 : False : False : Bi

In [10]:
print("Unassigned immediate: ", K - sum(pyo.value(m.immediate_v[k,j]) for k in range(K) for j in range(H)))
print("Assigned immediate: ", sum(pyo.value(m.immediate_v[k,j]) for k in range(K) for j in range(H)))
print("Unassigned delayed: ", L - sum(pyo.value(m.delayed_v[l,j]) for l in range(L) for j in range(H)))
print("Assigned delayed: ", sum(pyo.value(m.delayed_v[l,j]) for l in range(L) for j in range(H)))
print("Total bed: ", sum(bed_capacity_list))
print("Objective value: ", pyo.value(m.objective))

Unassigned immediate:  76.0
Assigned immediate:  267.0
Unassigned delayed:  1123.0
Assigned delayed:  576.0
Total bed:  843
Objective value:  3926404212.386926
